## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [4]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
yelp.sentiment.value_counts()
#positive = 1
#negative = 0

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [6]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [7]:
list(yelp.text)[0]

'Wow... Loved this place.'

In [8]:
# Answer below:
yelp['text_norm'] = yelp.text.apply(lambda x: normalize(x))
yelp.head()

#yelp.text.apply(normalize)

,text,sentiment,text_norm
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [9]:
# Answer below:
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = len(set(yelp['text_norm'].apply(lambda x: x.strip().split(' ')).sum())) 
vocab_size
#splitting individ words, putting into list then one long list then counting how many unique words

1629

In [ ]:
#len(set(yelp['text_norm'].sum().split(' ')))
#different approach to above be aware there's different amount

In [10]:
yelp['encoded'] = yelp['text_norm'].apply(one_hot, args=[vocab_size])
yelp.head()

,text,sentiment,text_norm,encoded
0,Wow... Loved this place.,1,wow love place,"[1015, 828, 930]"
1,Crust is not good.,0,crust not good,"[747, 15, 520]"
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti,"[15, 135, 1258, 1551]"
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...,"[610, 1047, 902, 656, 1163, 387, 192, 42, 828,..."
4,The selection on the menu was great and so wer...,1,select menu great price,"[1174, 583, 853, 1340]"


Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [12]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences

indep_vars = pad_sequences(yelp['encoded'])
indep_vars

array([[   0,    0,    0, ..., 1015,  828,  930],
       [   0,    0,    0, ...,  747,   15,  520],
       [   0,    0,    0, ...,  135, 1258, 1551],
       ...,
       [   0,    0,    0, ...,   15, 1341,  963],
       [   0,    0,    0, ..., 1587,  893,  914],
       [   0,    0,    0, ..., 1203, 1003,  660]], dtype=int32)

In [13]:
indep_vars.shape

(976, 83)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [14]:
# Answer below:
from sklearn.model_selection import train_test_split

target = yelp['sentiment']

X_train, X_test, y_train, y_test = train_test_split(indep_vars, target, test_size=0.2, random_state=42)

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [17]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

max_words = np.max(X_train) + 1

model = Sequential()
model.add(Embedding(max_words, 100, input_length=X_train.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 83, 100)           162800    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 167,329
Trainable params: 167,329
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [19]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fitting
model.fit(X_train, y_train,
          validation_data=(X_test, y_test), 
          batch_size=128,
          epochs=50)

Epoch 1/50
7/7 [==============================] - 2s 71ms/step - loss: 0.6929 - accuracy: 0.4980 - val_loss: 0.6910 - val_accuracy: 0.5510
Epoch 2/50
7/7 [==============================] - 0s 33ms/step - loss: 0.6669 - accuracy: 0.7305 - val_loss: 0.6901 - val_accuracy: 0.5204
Epoch 3/50
7/7 [==============================] - 0s 36ms/step - loss: 0.6409 - accuracy: 0.8363 - val_loss: 0.6859 - val_accuracy: 0.5408
Epoch 4/50
7/7 [==============================] - 0s 33ms/step - loss: 0.6074 - accuracy: 0.8864 - val_loss: 0.6814 - val_accuracy: 0.5714
Epoch 5/50
7/7 [==============================] - 0s 33ms/step - loss: 0.5564 - accuracy: 0.8880 - val_loss: 0.6770 - val_accuracy: 0.5816
Epoch 6/50
7/7 [==============================] - 0s 32ms/step - loss: 0.4880 - accuracy: 0.9120 - val_loss: 0.6823 - val_accuracy: 0.5561
Epoch 7/50
7/7 [==============================] - 0s 34ms/step - loss: 0.4413 - accuracy: 0.8696 - val_loss: 0.6674 - val_accuracy: 0.6020
Epoch 8/50
7/7 [===========

In [20]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 5ms/step - loss: 1.0882 - accuracy: 0.6480


[1.088241696357727, 0.6479591727256775]